In [ ]:
# Fetch the dataset from the csv using Pandas
import pandas as pd
import numpy as np

#list of columns to be read from the csv for the data part
# names = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"] 

dataset = pd.read_csv("diabetes.csv")

# Get the X
X = dataset.iloc[:, 0:8].values

#Get the labels
Y = dataset.iloc[:, 8].values


In [ ]:
# This function is used to convert the binary array into the corresponding decimal value using basic binary to decimal conversion. This is used to convert the af, hd1 and hd2 binary arrays to their corresponding decimal value
def binary_to_decimal(bin_str):
    dec_value = 0

    num_bits = 0 #stores the position of bit from right
    for bit in reversed(bin_str):
        if bit == 1:
            dec_value = dec_value + 2**num_bits
        num_bits = num_bits + 1
    
    return dec_value #return the decimal value of the binary array

# binary_to_decimal([0, 1, 1])


In [ ]:
#This function is used to convert the value of alpha from array to the float value using binary to decimal conversion with bits treated as after decimal.
def alpha_value(alpha):
    #Stores the value of the learning rate in form of float
    lr = 0.0

    num_bits = 1
    for i in range(len(alpha)):
        #if the value of alpha at position i is 1 then we can add the value to lr
        if alpha[i] == 1:
            lr = lr + (1/2**num_bits) #add the value as 1/2^r where r is the bit position from left
        num_bits = num_bits + 1

    return lr #return the value of the learning rate

# alpha_value([1, 0, 0, 1])


In [ ]:
#This function is used to split the chromosome into the lying components of the neural network. In general we will work on the chromosome as a whole, however for verifying and training we need the values of each component separately and thus, this returns all the components in form of list.
def split_chromosome(chromosome):
    #Length of the chromosome has to be 16
    assert len(chromosome) == 16

    alpha = chromosome[0:4] #alpha is the first 4 bits
    af = chromosome[4:6] #activation function is the next 2 bits
    hd1 = chromosome[6:11] #number of hidden neurons in hidden layer 1 are the next 5 bits
    hd2 = chromosome[11:16] #number of hidden neurons in hidden layer 2 are the next 5 bits

    return (alpha, af, hd1, hd2)

# print(split_chromosome([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]))

In [33]:
#function to verify if the chromosome is valid or not. Returns true if valid, ekse false
def verify_chromosome(chromo):
    #split the chromosome into the integer arrays for each component
    lr, af, hd1, hd2 = split_chromosome(chromo)

    #check the values of the components if satisfy the constraints.
    if alpha_value(lr) <= 0.0 or alpha_value(lr) >= 1.0:
        return False
    if binary_to_decimal(hd1) < 2:
        return False
    return True   #return True

In [ ]:
#generate_chromosome is responsible to generate a chromosome and sends true if the chromosome generated is a valid one (satisfies all the constraints) and false in case of invalid chromosome
def generate_chromosome(chromo_size):
    #generate the chromosome of size chromo_size with values between 0 or 1
    chromo = [np.random.randint(0, 2) for i in range(chromo_size)]

    flag = verify_chromosome(chromo)

    if flag:
        return (chromo, True)
    return (chromo, False)

# generate_chromosome(16)


In [ ]:
#Initial_Population is used to generate the initial population of size 30. The pop is generated by randomly setting each value as 0 or 1 for the 16 indices. We will also verify the value of each component simultaneously to ensure that all the components are in the constrains to be satisfied. This function returns a list of list of size (pop_size, chromosome_size)
def Initial_Population(pop_size, chromo_size):

    #Initialize the population variable as a list of lists of size (30, 16)
    pop = [[int() for i in range(chromo_size)] for j in range(pop_size)] 

    for i in range(pop_size):
        #set the flag as false
        flag = False

        while not flag:
            #get the values of the chromosome and flag from the function. If flag becomes true then the while loop stops and generates no more chromosomes.
            pop[i], flag = generate_chromosome(chromo_size)
    
    return pop

# Initial_Population(30, 16)



In [ ]:
# Dictionary for mapping the activation functions to the integer values
int_af = {}

int_af[0] = 'relu'
int_af[1] = 'tanh'
int_af[2] = 'sigmoid'
int_af[3] = 'selu'

In [ ]:
# Model Params

#population size
pop_size = 30

#chromosome size
chromo_size = 16

#number of generations
num_gen = 100

#number of epochs for training
num_epochs = 25

#batch size
batch_size = 32

In [ ]:
#import the dependencies
import keras
from tensorflow.keras.initializers import GlorotUniform as glorot_uniform
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential

#Train_model is used to train the model and return the loss after the training. For this, first convert the input chromosome into it's components by splitting the chromosome into the values and then constructing the neural network and then training for the number of epochs = 25.
input_shape = X.shape #input shape
classes = 1 #consist of only 1 label true or false

def Train_model(chromo):


    #split the chromosome into the components
    lr, af, hd1, hd2 = split_chromosome(chromo)

    alpha = alpha_value(lr) #convert the integer array to the float value
    act_func = int_af[binary_to_decimal(af)] #get the activation function string using the map
    h1 = binary_to_decimal(hd1) #number of hidden neurons layer 1
    h2 = binary_to_decimal(hd2) #number of hidden neurons layer 2

    #Construct the model
    model = Sequential()

    #first hidden layer
    model.add(Dense(h1, input_shape = (input_shape[1],), activation = act_func, kernel_initializer = glorot_uniform(seed = 0)))

    #second hidden layer
    model.add(Dense(h2, activation = act_func, kernel_initializer = glorot_uniform(seed = 0)))

    #output layer
    model.add(Dense(classes, activation = 'sigmoid'))
    
    #Optimizer used for the training
    opt = keras.optimizers.Adam(learning_rate=alpha)
    model.compile(loss='binary_crossentropy', optimizer=opt)

    #call the fit method to perform the training. Store the history used to get the lossesx
    history = model.fit(x = X, y = Y, epochs = num_epochs, verbose = 0)
    losses = history.history['loss']

    #return the mean loss
    return np.mean(losses)

# Train_model([0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0])


In [ ]:
from tqdm import tqdm #for the progress bar

# --------------------------- MAIN FUNCTION ---------------------------------
#This function is responsible to handle the entire algorithm. It takes as arguments the population size, chromosome size and the number of generations or number of epochs and returns the minimum loss or maximum fitness for each generation or population.

def main(pop_size, chromo_size, num_gen):
    #generate the initial population
    pop = Initial_Population(pop_size, chromo_size)

    #stores the losses for entire generation
    losses = [float() for i in range(pop_size)]

    #stores the fitness for entire generation
    fitness = [float() for i in range(pop_size)]

    #run a loop for the number of generations
    for i in range(num_gen):
        title = "Generation {} Progress".format(i + 1)

        # Train the model for different generations and the metric for fitness is the loss. Lower the loss, higher the fitness
        for j in tqdm(range(pop_size), title, ncols = 100):
            losses[j] = Train_model(pop[j])

            #fitness is the inverse of the loss.
            fitness[j] = 1.0/losses[j]
        
             
    
main(pop_size, chromo_size, 1)
